In [1]:
%run 00_setup.ipynb

23/11/21 13:41:07 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [45]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sequence, explode, to_date, datediff, monotonically_increasing_id, avg

In [35]:
# more than 7 days stay
over7d_notGrupped = ex_df.filter(datediff(to_date("srch_co"), to_date("srch_ci")) > 7)
over7d = over7d_notGrupped.groupBy('hotel_id', 'srch_ci', 'srch_co').count()
# add stay_id here
over7d = over7d.withColumn('stay_id', monotonically_increasing_id())

In [55]:
# expand date range to records with exact dates and year_month field
over7 = over7d.select(
    'stay_id',
    'hotel_id',
    'srch_ci',
    'srch_co',
    explode(sequence(to_date('srch_ci'),to_date('srch_co'))).alias('date')
)
over7.count()

679196

In [54]:
# joid weather data, on hotel_id cover only 5% of records in "stay over 7days" data. Must find workaround

over7weather = over7.join(hw_df, [over7.hotel_id == hw_df.id, over7.date == hw_df.wthr_date] , "inner")
over7weather.count()

33280

In [59]:
# let's update over7 with geohash and join weather data on geohash and date.
over7geo = over7.join(hw_df, over7.hotel_id == hw_df.id, "inner").select(over7['*'], hw_df.geoHash)
over7weather = over7geo.join(hw_df, [over7geo.geoHash == hw_df.geoHash, over7geo.date == hw_df.wthr_date] , "inner").select(over7geo['*'], hw_df.avg_tmpr_c)
over7weather.count()

AnalysisException: Column avg_tmpr_c#1 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.

In [41]:
over7weather.printSchema()

root
 |-- stay_id: long (nullable = false)
 |-- hotel_id: long (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- date: date (nullable = false)
 |-- address: string (nullable = true)
 |-- avg_tmpr_c: double (nullable = true)
 |-- avg_tmpr_f: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- geoHash: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- wthr_date: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [51]:
# temperature difference last and first day of stay) and, average temperature during stay
window = Window.partitionBy('stay_id').orderBy('date')
result = over7weather.withColumn("avg_tmpr_stay", avg('avg_tmpr_c').over(window))
result.select(
    'stay_id',
    'hotel_id',
    'srch_ci',
    'srch_co',
    'date',
    'avg_tmpr_stay'
).where('hotel_id = 824633720836').show(100)

+-------+------------+----------+----------+----------+-------------+
|stay_id|    hotel_id|   srch_ci|   srch_co|      date|avg_tmpr_stay|
+-------+------------+----------+----------+----------+-------------+
|      1|824633720836|2017-09-12|2017-10-28|2017-09-20|         23.4|
|      1|824633720836|2017-09-12|2017-10-28|2017-09-27|         23.5|
|   1720|824633720836|2017-08-17|2017-08-28|2017-08-27|         17.6|
|   1987|824633720836|2017-09-21|2017-10-02|2017-09-27|         23.6|
|  15676|824633720836|2017-08-24|2017-09-02|2017-08-27|         17.6|
|  28339|824633720836|2017-08-17|2017-08-31|2017-08-27|         17.6|
|  30454|824633720836|2017-08-23|2017-08-31|2017-08-27|         17.6|
|  31707|824633720836|2017-09-18|2017-09-26|2017-09-20|         23.4|
|  32187|824633720836|2017-09-16|2017-09-26|2017-09-20|         23.4|
|  48791|824633720836|2017-08-22|2017-08-30|2017-08-27|         17.6|
|  49459|824633720836|2016-10-22|2016-10-30|2016-10-25|          7.7|
|  53072|82463372083

In [53]:
ex_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- date_time: string (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- hotel_id: long (nullable = true)

